In [1]:
# Importing the required packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm

import os
import numpy as np
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# Global variables
MAX_CIFAR = 1000
MAX_TRAIN, MAX_VAL, MAX_TEST = 6500, 3500, 2500 
final_img_size = (96, 96, 3)
MULTIPLIER = int(final_img_size[0]/32)

In [3]:
# Helper Modules
def get_random_data(data1, data2, low, high, max_samples=100, multiplier=1):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.empty((max_samples, H1*multiplier, W1*multiplier, C1))
    suff_data2 = np.empty((max_samples, N))
    shuffles = np.random.randint(low, high, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = rescale(data1[shuffles[idx], :, :, :], multiplier, anti_aliasing=False)
        suff_data2[idx] = data2[shuffles[idx], :]
    return suff_data1, suff_data2

def load_formatted_image(input_dir, limit=None, img_size=final_img_size):
    all_files = os.listdir(input_dir)
    if limit is not None:
        np.random.shuffle(all_files)
        all_files = all_files[:limit]
    img_arr = np.zeros((len(all_files), img_size[0], img_size[1], img_size[2]))
    for idx in range(len(all_files)):
        img_arr[idx] = resize(plt.imread(os.path.join(input_dir, all_files[idx])), img_size, anti_aliasing=False)
        
    return img_arr

In [4]:
# Loading CIFAR dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()
X, y = np.concatenate((x_train, x_test), axis=0), np.concatenate((y_train, y_test), axis=0)
res_X, _ = get_random_data(X, y, 0, X.shape[0], MAX_CIFAR, multiplier=MULTIPLIER)

# Loading KAIST and Soda dataset
kaist_arr = load_formatted_image('../data/KAIST/')
soda_arr = load_formatted_image('../data/all_soda_bottles/')
caltech_handpicked = load_formatted_image('../data/101_handpicked/')

In [5]:
# Merging KAIST and CIFAR dataset and creating training, test splits
y_others = np.zeros(((kaist_arr.shape[0]+res_X.shape[0]), 1))
y_soda = np.ones((soda_arr.shape[0], 1))

pre_X, pre_y = np.concatenate((soda_arr, kaist_arr, res_X)), np.concatenate((y_soda, y_others))
buff_X, buff_y = get_random_data(pre_X, pre_y, 0, pre_X.shape[0], pre_X.shape[0])

# Weighing in the Caltech Object Dataset
X = np.concatenate((buff_X[:(MAX_TRAIN+MAX_VAL+MAX_TEST+100)], caltech_handpicked))
y = np.concatenate((buff_y[:X.shape[0]], np.zeros((caltech_handpicked.shape[0], 1))))

X_train, y_train = X[:MAX_TRAIN], y[:MAX_TRAIN]
X_val, y_val = X[MAX_TRAIN:MAX_TRAIN+MAX_VAL], y[MAX_TRAIN:MAX_TRAIN+MAX_VAL]
X_test, y_test = X[MAX_TRAIN+MAX_VAL:MAX_TRAIN+MAX_VAL+MAX_TEST], y[MAX_TRAIN+MAX_VAL:MAX_TRAIN+MAX_VAL+MAX_TEST]

In [6]:
# Model hyperparameters
activation_func = 'selu'
weight_decay = 1e-5
curr_optimizer = 'Adam'

In [7]:
# Proposed variant of the LeNet-5 architecture
model = Sequential()

#Instantiating Layer 1
model.add(Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                 kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))
model.add(BatchNormalization())

# #Instantiating Layer 2
model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))
model.add(BatchNormalization())

# #Instantiating Layer 3
model.add(Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Flatten())

#Instantiating Layer 4
model.add(Dense(84, activation=activation_func)) 

#Output Layer
model.add(Dense(1, activation='sigmoid'))    

model.compile(loss="binary_crossentropy", optimizer=curr_optimizer, validation_data=(X_val, y_val),
                  metrics=["accuracy", "mae", "mse"])

In [8]:
model.fit(X_train, y_train, epochs=12, batch_size=512)

Train on 6500 samples
Epoch 1/12
6500/6500 [==============================] - 24s 4ms/sample - loss: 0.9515 - accuracy: 0.9126 - mae: 0.0873 - mse: 0.0674
Epoch 2/12
6500/6500 [==============================] - 21s 3ms/sample - loss: 0.0232 - accuracy: 0.9943 - mae: 0.0083 - mse: 0.0041
Epoch 3/12
6500/6500 [==============================] - 22s 3ms/sample - loss: 0.0038 - accuracy: 0.9995 - mae: 0.0014 - mse: 5.4578e-04
Epoch 4/12
6500/6500 [==============================] - 21s 3ms/sample - loss: 0.0017 - accuracy: 0.9998 - mae: 4.4920e-04 - mse: 2.0691e-04
Epoch 5/12
6500/6500 [==============================] - 21s 3ms/sample - loss: 6.4570e-04 - accuracy: 1.0000 - mae: 2.1322e-04 - mse: 3.0125e-05
Epoch 6/12
6500/6500 [==============================] - 21s 3ms/sample - loss: 6.1733e-04 - accuracy: 1.0000 - mae: 1.8907e-04 - mse: 2.3465e-05
Epoch 7/12
6500/6500 [==============================] - 22s 3ms/sample - loss: 5.2008e-04 - accuracy: 1.0000 - mae: 1.0263e-04 - mse: 4.9092e-06

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=100)
score

In [ ]:
# coca_cola_can.jpg  coke_can.jpg  coke_can2.jpg  coke_can3.jpg  coke_can4.jpg  milk_room.jpg  vodka.jpg  final_img_size.jpg

In [ ]:
# # Load and evaluate Test set 
# im = resize(plt.imread('../data/manual_coke_set/vodka.jpg'), final_img_size, anti_aliasing=False)
# plt.imshow(im)
# imz = im[np.newaxis]

# y_pred = model.predict(imz)
# y_pred